In [25]:
from dotenv import load_dotenv
import plotly.express as px
import plotly.graph_objects as go

from app.cli_args import get_database_path
from infra.db_connection import db_connect
from domain.calc_profitability import calc_balance_for_all_accounts
from app.best_csp_savings_account import find_best_csp_savings_account

load_dotenv()
db_path = get_database_path()
conn = db_connect(db_path)
df = calc_balance_for_all_accounts(conn)
best_csp_account = find_best_csp_savings_account(conn)
best_csp_account

account_id                                                15.0
mean                                                  4.063571
std                                                   0.189259
csp                                                  21.470995
account_name    Goldman Sachs Bank USA Cash Management Account
Name: 14, dtype: object

In [3]:
df

,accountId,accountName,balance
0,1,Bank of America High-Yield Savings Account,1042.155938
1,2,Bank of America Money Market Account,1043.203640
2,3,Bank of America Cash Management Account,1038.222307
3,4,Chase Bank High-Yield Savings Account,1041.557269
4,5,Chase Bank Money Market Account,1041.744104
5,6,Chase Bank Cash Management Account,1040.570507
6,7,Wells Fargo High-Yield Savings Account,1042.115873
7,8,Wells Fargo Money Market Account,1039.632901
8,9,Wells Fargo Cash Management Account,1039.885813
9,10,Ally Bank High-Yield Savings Account,1041.097909


In [4]:
max_balance_account = df.loc[df['balance'].idxmax()]
min_balance_account = df.loc[df['balance'].idxmin()]
print('max_balance_account', max_balance_account)
print('min_balance_account', min_balance_account)

max_balance_account accountId                                            22
accountName    Discover Bank High-Yield Savings Account
balance                                     1044.390011
Name: 21, dtype: object
min_balance_account accountId                                            3
accountName    Bank of America Cash Management Account
balance                                    1038.222307
Name: 2, dtype: object


## Account Balances Bar Chart

In [27]:

df_sorted = df.sort_values(by='balance', ascending=False)
df_sorted

default_colors = px.colors.qualitative.Plotly
highlight_color = default_colors[1]
normal_color = default_colors[0]

colors = [highlight_color if account == best_csp_account['account_id'] else normal_color for account in df_sorted['accountId']]

fig = go.Figure()

for account, balance, color in zip(df_sorted['accountName'], df_sorted['balance'], colors):
    fig.add_trace(go.Bar(
        x=[account],
        y=[balance],
        name=account,
        marker_color=color
    ))

fig.update_layout(
    xaxis_tickangle=-90,
    xaxis_tickfont=dict(size=10),
    height=800,
    width=1000,
    yaxis=dict(range=[df['balance'].min()-1, df['balance'].max()]),
    showlegend=False
)

fig.show()
